In [2]:
from tensorflow.python.util.tf_export import InvalidSymbolNameError
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
import numpy as np
import pandas as pd
import time
from pathlib import Path
from sklearn import metrics
import os
import tensorflow_probability as tfp

In [3]:
# auto search files in specific file path
dicnpy ='../dataUNSW/' #获取当前路径  
for root,dirs,files in os.walk(dicnpy):  #遍历统计
	for each in files: 
		_,filesuffix= os.path.splitext(each)
		if filesuffix=='.npy' or filesuffix=='.pdf':
			os.remove(os.path.join(root, each))
		#print(dicnpy+each)

# Data preprocessing; Custom tokens
manualy transform non-numeric colums to numeric columes, including columes of 'proto','service','state','attack_cat'. Because in this dataset, the options of protocol type is different between train set to test set, so that the one-hot embedding could not be used.
<br>
in this way, each distinct value is set to a integer, it is not exactly right for the outcome as the relationship of values is changed.

In [4]:
# prepare columns headers
features=pd.read_csv('../Datasets/NUSW-NB15_features.csv',usecols=[0,1],header=0,encoding='cp1252')
strheader=[ i for i in features.iloc[:,1]]


In [5]:
'''
split data into group including normal, each type of attack.
capture normal group with numeric type and append save in npy file. the target of transferting attack_cat to numeric is for multi labels classifciation
'''
def savegroup(data_num):
    normal_train=data_num[data_num.attack_cat=="Normal"].copy()
    normal_train.loc[normal_train.attack_cat=="Normal","attack_cat"]=0
    p=Path(dicnpy+'Normal.npy')
    if p.exists():
        old=np.load(dicnpy+'Normal.npy', allow_pickle=True)
        new_record=np.append(old,normal_train,0)
        print(new_record.shape)
        np.save(dicnpy+'Normal.npy',new_record)
    else:
        np.save(dicnpy+'Normal.npy',normal_train)

    # capture each attack group in numeric type, save in the dic of train_group.
    #train_group={"Normal":normal_train}
    group_types=data_num["attack_cat"].value_counts()
    print(group_types)
    ite=1
    for ind, ptype in group_types.items():
        single_group=data_num[data_num.attack_cat==ind].copy()  
        if ind.strip()!='Normal':
            single_group.loc[single_group.attack_cat==ind, "attack_cat"]=ite
            # save in npy file
            p=Path(dicnpy+ind.strip()+'.npy')
            if p.exists():
                old=np.load(dicnpy+ind.strip()+'.npy', allow_pickle=True)
                print(old.shape)
                new_record=np.append(old,single_group.sample(frac=1),0)
                np.save(dicnpy+ind.strip()+'.npy',new_record)
            else:
                np.save(dicnpy+ind.strip()+'.npy',single_group.sample(frac=1))
        else:
            continue         
        #train_group[ind]=temp.sample(frac=1)
        ite=ite+1

#print((train_group['Generic'].shape) )

In [6]:
# there are many 0x data in 'sport' and 'dsport' and other columns, so tranfer them to numeric.
def str2num(inivalue):
    if isinstance(inivalue,int):
        return inivalue
    elif isinstance(inivalue,str):
        if inivalue.isnumeric():
            return int(inivalue)
        else:
            if inivalue.startswith('0x',0):
                return int(inivalue,16)
            elif inivalue=='-' or inivalue.strip()=='':
                return 0
            else:
                print(inivalue)
                return 0
    
    else:
        print(inivalue)
        return 0

In [7]:
# start reading data
# read 4 csv files, 559M
files=["UNSW-NB15_1.csv","UNSW-NB15_2.csv","UNSW-NB15_3.csv","UNSW-NB15_4.csv"]
dicdata='../Datasets/'
dicnpy='../dataUNSW/'
ip2int=lambda x:sum([256**j*int(i) for j,i in enumerate(x.split('.')[::-1])])
#int2ip= lambda x: '.'.join([str(x/(256**i)%256) for i in range(3,-1,-1)])
state_types={1:"no", 2:"URN", 3:"ACC", 4:"CLO", 5:"CON", 6:"ECO", 7:"ECR", 8:"URH", 9:"INT", 10:"MAS", 11:"PAR", 12:"REQ",  13:"TST", 14:"TXD", 15:"FIN", 16:"RST"} 
start=time.time()

for i in range(0, len(files)):
    filedic=dicdata+files[i]
    print(filedic)
    train_data=pd.read_csv(filedic, header=None,skip_blank_lines='true')  #these 4 CSV files have not header line.
    train_data.columns=strheader
    train_data['attack_cat'].fillna('Normal',inplace=True)
    train_data.fillna(0,inplace=True)
    train_num=train_data.copy()

    # numeric non-numeric columns with the order. only the column of "state"       
    print(len(state_types))
    for inde in range(1, len(state_types)+1):
        train_num.loc[train_num["state"]==str(state_types[inde]), "state"]=inde

    # numeric non-numeric columns except the column of attack_cat. 
    str_columns=["service","proto"] #,"attack_cat"]    
    for sc in str_columns:
        items_set=train_num[sc].value_counts()
        temp=1
        for ind, ptype in items_set.items():
            train_num.loc[train_num[sc]==ind, sc]=temp
            temp=temp+1

    # transfer IP address to numeric.
    if type(train_num.at[3,'srcip'])!=np.int64 :
        train_num['srcip']=train_num['srcip'].apply(ip2int)
        train_num['dstip']=train_num['dstip'].apply(ip2int)

    # check out if columns be the numeric type. and transfer them. must be the end of  processing.
    for i in strheader[:-2]:
        try:
            np.asarray(train_num[i]).astype(np.float32)
        except ValueError:
            numseries=[]
            for ind ,cont in train_num.iterrows():
                numseries.append(str2num(cont[i]))
            train_num[i]=numseries

    # save in npy file according attack types.
    savegroup(train_num)

    #break
print(train_data.shape)
end=time.time()
print(f"\nTime to read: {round(end-start,5)} seconds.")

../venv/Datasets/UNSW-NB15_1.csv


/tmp/ipykernel_2848559/327677390.py:14: DtypeWarning: Columns (1,3,47) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data=pd.read_csv(filedic, header=None,skip_blank_lines='true')  #these 4 CSV files have not header line.


16
Normal            677786
Generic             7522
Exploits            5409
 Fuzzers            5051
Reconnaissance      1759
DoS                 1167
Backdoors            534
Analysis             526
Shellcode            223
Worms                 24
Name: attack_cat, dtype: int64
../venv/Datasets/UNSW-NB15_2.csv


/tmp/ipykernel_2848559/327677390.py:14: DtypeWarning: Columns (3,39,47) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data=pd.read_csv(filedic, header=None,skip_blank_lines='true')  #these 4 CSV files have not header line.


16
(1325038, 49)
Normal             647252
Generic             27883
Exploits            11103
 Fuzzers             4668
DoS                  4637
 Reconnaissance      3116
Analysis              608
Backdoors             370
 Shellcode            324
Worms                  40
Name: attack_cat, dtype: int64
(7522, 49)
(5409, 49)
(5051, 49)
(1167, 49)
(1759, 49)
(526, 49)
(534, 49)
(223, 49)
(24, 49)
../venv/Datasets/UNSW-NB15_3.csv
16
(1867614, 49)
Normal             542576
Generic            118198
Exploits            16574
 Fuzzers             9137
DoS                  5642
 Reconnaissance      5582
Analysis              873
Backdoors             759
 Shellcode            593
Worms                  67
Name: attack_cat, dtype: int64
(35405, 49)
(16512, 49)
(9719, 49)
(5804, 49)
(4875, 49)
(1134, 49)
(904, 49)
(547, 49)
(64, 49)
../venv/Datasets/UNSW-NB15_4.csv
16
(2218764, 49)
Normal             351150
Generic             61878
Exploits            11439
 Fuzzers             5390
DoS   